# Chapter 7 - Can I have some of my favorite queries and a distributed database?

## MongoDB

Paul E. Anderson

## Ice Breaker

Favorite mac and cheese recipe or spin off?

While this text can be viewed as PDF, it is most useful to have a Jupyter environment. I have an environment ready for each of you, but you can get your own local environment going in several ways. One popular way is with Anaconda (<a href="https://www.anaconda.com/">https://www.anaconda.com/</a>. Because of the limited time, you can use my server.

### Update on lab grades

<img src='../labs/grade_summary.png' width=600>

## Hu(mongo)us

Used by companies such as Foursquare, bit.ly, and CERN for collecting Large Hadron Collider data.

Document database (JSON) ... though technically data is stored as BSON (binary form of JSON).

Allows data to persist in a nest state

Can query nested data in an ad hoc fashion

## What have we lost from relational databases?
We can no longer perform server side joins. 

## What's in an ID field?
Each document in a collection gets an automatic ObjectID:
* 12 bytes
* composed of a timestamp + client machine ID + client process ID + a 3-byte incremented counter

Why is this interesting? 
* Autonumbering scheme allows each process on every machine to handle its own ID generation without colliding with other instances.

## Let's have some fun with Mongo

We will use PyMongo, but you can use the native Javascript interface at the command line. We need to connect to our database (``csc-369``), and a collection of daily COVID19 cases as described in the lab.

In [2]:
from pymongo import MongoClient
client = MongoClient()

db = client["csc-369"]
col = db["daily"]

col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'csc-369'), 'daily')

### Finding one document

In [4]:
col.find_one()

{'_id': ObjectId('60392e3656264fee961ca816'),
 'date': 20200405,
 'state': 'AK',
 'positive': 185,
 'negative': 6099,
 'pending': None,
 'hospitalizedCurrently': None,
 'hospitalizedCumulative': 20,
 'inIcuCurrently': None,
 'inIcuCumulative': None,
 'onVentilatorCurrently': None,
 'onVentilatorCumulative': None,
 'recovered': None,
 'hash': '661d7b0f627847a2dceb5d70d4e9260965031cc2',
 'dateChecked': '2020-04-05T20:00:00Z',
 'death': 6,
 'hospitalized': 20,
 'total': 6284,
 'totalTestResults': 6284,
 'posNeg': 6284,
 'fips': '02',
 'deathIncrease': 1,
 'hospitalizedIncrease': 4,
 'negativeIncrease': 230,
 'positiveIncrease': 14,
 'totalTestResultsIncrease': 244}

### Finding that document again

In [6]:
list(col.find({"_id": '60392e3656264fee961ca816'}))

[]

### What happened?

In [10]:
from bson.objectid import ObjectId
list(col.find({"_id": ObjectId('60392e3656264fee961ca816')}))

[{'_id': ObjectId('60392e3656264fee961ca816'),
  'date': 20200405,
  'state': 'AK',
  'positive': 185,
  'negative': 6099,
  'pending': None,
  'hospitalizedCurrently': None,
  'hospitalizedCumulative': 20,
  'inIcuCurrently': None,
  'inIcuCumulative': None,
  'onVentilatorCurrently': None,
  'onVentilatorCumulative': None,
  'recovered': None,
  'hash': '661d7b0f627847a2dceb5d70d4e9260965031cc2',
  'dateChecked': '2020-04-05T20:00:00Z',
  'death': 6,
  'hospitalized': 20,
  'total': 6284,
  'totalTestResults': 6284,
  'posNeg': 6284,
  'fips': '02',
  'deathIncrease': 1,
  'hospitalizedIncrease': 4,
  'negativeIncrease': 230,
  'positiveIncrease': 14,
  'totalTestResultsIncrease': 244}]

### Ad hoc queries can be simple (above) or complex

In [31]:
col.count_documents(
    {'state': {"$regex": "^A"},
     'totalTestResults': {"$gt": 6000}
    }
)

26

### What about a little OR?

In [32]:
col.count_documents(
    {
        "$or": [
            {'state': {"$regex": "^A"}},
            {'totalTestResults': {"$gt": 6000}}
        ]
    }
)

575

| Command      | Description |
| ------------ | ----------- |
| \$regex       | Match by any PCRE-compliant regular expression string |
| \$ne          | Not equal to        |
| \$lt          | Less than |
| \$lte         | Less than or equal to |
| \$gt          | Greater than |
| \$gte         | Greater than or equal to |
| \$exists      | Check for the existence of a field |
| \$all         | Match all elements in an array |
| \$in          | Match any elements in an array |
| \$nin         | Does not match any elements in an array |
| \$elemMatch    | match all fields in an array of nested documents |
| \$or | or |
| \$nor         | Not or |
| \$size        | Match array of given size |
| \$mod         | Modulus |
| \$type        | Match if field is a given datatype |
| \$not         | Negate the given operator check |

### What if we wanted to update a mispelling?

In [39]:
col.update_one(
    {"_id": ObjectId('60392e3656264fee961ca816')},
    {"$set": {"state":"ARK"}})

In [40]:
list(col.find({"_id": ObjectId('60392e3656264fee961ca816')}))

[{'_id': ObjectId('60392e3656264fee961ca816'), 'state': 'ARK'}]

In [41]:
col.update_one(
    {"_id": ObjectId('60392e3656264fee961ca816')},
    {"$set": {"state":"AK"}})

In [42]:
list(col.find({"_id": ObjectId('60392e3656264fee961ca816')}))

[{'_id': ObjectId('60392e3656264fee961ca816'), 'state': 'AK'}]

| Command      | Description |
| ------------ | ----------- |
| \$set | Sets the given field with the given value |
| \$unset | Removes the field |
| \$inc | Adds the given field by the given number |
| \$pop | Removes the last (or first) element from an array |
| \$push | Adds the value to an array |
| \$pushAll | Adds all values to an array |
| \$addToSet | Similar to push, but won’t duplicate values |
| \$pull | Removes matching value from an array |
| \$pullAll | Removes all matching values from an array |

### Warning

Mongo will let you do what you want. Even the following:

In [43]:
col.update_one(
    {"_id": ObjectId('60392e3656264fee961ca816')},
    {"$set": {"state":123}})

In [44]:
list(col.find({"_id": ObjectId('60392e3656264fee961ca816')}))

[{'_id': ObjectId('60392e3656264fee961ca816'), 'state': 123}]

In [45]:
col.update_one(
    {"_id": ObjectId('60392e3656264fee961ca816')},
    {"$set": {"state":"AK"}})

## Plug for more database classes
By default PyMongo creates an index to help you search for specific documents by ID, but it does not do so on other fields. You can create an
index for faster queries using ``ensureIndex``. This can change your queries from minutes to seconds or even milliseconds.

## Aggregation in MongoDB

The aggregate functionality in Mongo is extensive (https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/). We will cover some of the most common functionality such as matching and grouping.

In [61]:
result = col.aggregate([
    # Match the documents possible
    { "$match": { "totalTestResults": { "$gt": 6000 } } },

    # Group the documents and "count" via $sum on the values
    { "$group": {
        "_id": { # key
            "state": "$state"
        },
        "count": { "$sum": 1 } # action
    }}
])
import pprint
pprint.pprint(list((result)))

[{'_id': {'state': 'ME'}, 'count': 6},
 {'_id': {'state': 'AR'}, 'count': 6},
 {'_id': {'state': 'OH'}, 'count': 12},
 {'_id': {'state': 'IA'}, 'count': 7},
 {'_id': {'state': 'MS'}, 'count': 3},
 {'_id': {'state': 'LA'}, 'count': 13},
 {'_id': {'state': 'KY'}, 'count': 7},
 {'_id': {'state': 'MA'}, 'count': 15},
 {'_id': {'state': 'KS'}, 'count': 4},
 {'_id': {'state': 'MD'}, 'count': 9},
 {'_id': {'state': 'CO'}, 'count': 13},
 {'_id': {'state': 'NH'}, 'count': 5},
 {'_id': {'state': 'IL'}, 'count': 16},
 {'_id': {'state': 'GA'}, 'count': 12},
 {'_id': {'state': 'AK'}, 'count': 2},
 {'_id': {'state': 'IN'}, 'count': 10},
 {'_id': {'state': 'ID'}, 'count': 5},
 {'_id': {'state': 'AZ'}, 'count': 10},
 {'_id': {'state': 'AL'}, 'count': 7},
 {'_id': {'state': 'NV'}, 'count': 10},
 {'_id': {'state': 'PA'}, 'count': 14},
 {'_id': {'state': 'WA'}, 'count': 23},
 {'_id': {'state': 'CA'}, 'count': 21},
 {'_id': {'state': 'HI'}, 'count': 8},
 {'_id': {'state': 'UT'}, 'count': 12},
 {'_id': {'s

## MapReduce MongoDB

In order to do this, we will need some Javascript. It's pretty simple if you know languages such as Python, Java, C. We will perform the same aggregation as we did above.

In [66]:
from bson.code import Code
map_func = Code(
"""
function () {
    if (this.totalTestResults > 6000) {
        return emit(this.state, 1);
    }
}
"""
)

reduce_func = Code(
"""
function(key,values) {
    var total = 0;
    for (var i = 0; i < values.length; i++) {
        total += values[i];
    }
    return total;
}
"""
)

result = col.map_reduce(map_func, reduce_func, "myresults")
for doc in result.find():
    print(doc)

{'_id': 'AK', 'value': 2.0}
{'_id': 'AL', 'value': 7.0}
{'_id': 'AR', 'value': 6.0}
{'_id': 'AZ', 'value': 10.0}
{'_id': 'CA', 'value': 21.0}
{'_id': 'CO', 'value': 13.0}
{'_id': 'CT', 'value': 11.0}
{'_id': 'DC', 'value': 2.0}
{'_id': 'DE', 'value': 2.0}
{'_id': 'FL', 'value': 16.0}
{'_id': 'GA', 'value': 12.0}
{'_id': 'HI', 'value': 8.0}
{'_id': 'IA', 'value': 7.0}
{'_id': 'ID', 'value': 5.0}
{'_id': 'IL', 'value': 16.0}
{'_id': 'IN', 'value': 10.0}
{'_id': 'KS', 'value': 4.0}
{'_id': 'KY', 'value': 7.0}
{'_id': 'LA', 'value': 13.0}
{'_id': 'MA', 'value': 15.0}
{'_id': 'MD', 'value': 9.0}
{'_id': 'ME', 'value': 6.0}
{'_id': 'MI', 'value': 11.0}
{'_id': 'MN', 'value': 12.0}
{'_id': 'MO', 'value': 9.0}
{'_id': 'MS', 'value': 3.0}
{'_id': 'MT', 'value': 2.0}
{'_id': 'NC', 'value': 15.0}
{'_id': 'ND', 'value': 2.0}
{'_id': 'NH', 'value': 5.0}
{'_id': 'NJ', 'value': 13.0}
{'_id': 'NM', 'value': 12.0}
{'_id': 'NV', 'value': 10.0}
{'_id': 'NY', 'value': 20.0}
{'_id': 'OH', 'value': 12.0}
{'

## Wrap-up
We have introduced advanced functionality of Mongo including aggregation, filtering, counting, and map-reduce.